In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import PercentFormatter
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series, yield_curve, from_iso, yield_curves_pw, yield_curve_rv
from bgs.linker_analytics import linker_real_yield

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import PercentFormatter
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series, yield_curve, from_iso, yield_curves_pw, yield_curve_rv
from bgs.linker_analytics import linker_real_yield

%load_ext autoreload
%autoreload 2

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)

### Base Rate

In [ ]:
boe_base_rate = pd.read_csv("downloads/Bank Rate history and data  Bank of England Database.csv")
boe_base_rate.head()
boe_base_rate['Date Changed'] = pd.to_datetime(boe_base_rate['Date Changed'], format="%d %b %y", errors='coerce')
base_rate = boe_base_rate.copy().set_index('Date Changed').resample('D').ffill().reset_index().set_index('Date Changed')
base_rate.index = pd.to_datetime(base_rate.index)

In [ ]:
all_conv = conv_details[['Sequence', 'ISIN Code','%','Latest redemption date', 'Issue date', 'First coupon payable on date']]


In [ ]:
month_end_dates = pd.date_range(start='2024-01-31', end='2025-04-30', freq='ME').strftime("%Y-%m-%d").to_list()

In [ ]:
def bgs_gilt_yield(bgs_id, calc_date):

    bond = conv_details[conv_details['Sequence']==bgs_id]
    issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
    maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
    last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    coupon=bond['%'].values[0]/100

    clean_price=price_df.loc[calc_date,bgs_id]


    return gilt_yield(
        trade_date=calc_date,
        issue_date=issue_date,
        maturity_date=maturity_date,
        first_cpn_date=first_cpn_date,
        last_cpn_date=last_cpn_date,
        clean_price=clean_price,
        coupon=coupon)

In [ ]:
dislocations = []
starting = all_conv.copy()
calculation_date = month_end_dates[0]

gilts_list = starting.loc[(starting['Latest redemption date'] > pd.to_datetime(calculation_date))&(starting['Issue date']< pd.to_datetime(calculation_date))]
gilts_list['Amount_Outstanding']=gilts_list['Sequence'].map( lambda x: conv.loc[calculation_date, x])
gilts_list['price'] = gilts_list['Sequence'].apply(lambda x: price_df[x].loc[calculation_date])

In [ ]:
X = ((gilts_list['Latest redemption date'] - pd.Timestamp("1970-01-01")).dt.days).to_list()
# X = [np.array(X)[:, np.newaxis]]
X = np.array(X).reshape(-1, 1)
y = gilts_list['price']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVR

pipeline = make_pipeline(
    preprocessing.StandardScaler(),
    SVR(kernel='linear', epsilon=0.01, C=100, gamma = 0.01),
)

In [ ]:
pipeline.fit(X, y)

In [ ]:
gilts_list['decision_line'] = pipeline.predict(X)

In [ ]:
condition = (gilts_list['decision_line'] - gilts_list['price']) > 2
gilts_list['segmentation'] = condition.apply(lambda x: 1 if x else 0)

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="price",
    size="Amount_Outstanding",
    hue="segmentation",
    legend=False,
    sizes=(10, 500)
)

# show the graph
ax.plot(gilts_list['Latest redemption date'], gilts_list['decision_line'])
plt.show()

In [ ]:
gilts_list['yield'] = gilts_list['Sequence'].apply(lambda x: bgs_gilt_yield(bgs_id=x, calc_date=calculation_date))

c = yield_curve(calculation_date, gilts_list, method="Svensson")
dates = [from_iso(x) for x in [x.strftime("%Y-%m-%d") for x in gilts_list['Latest redemption date'].to_list()]]


day_count = ql.ActualActual(ql.ActualActual.Bond)

gilts_list['curve_yield'] = [
        c.zeroRate(d, day_count, ql.Continuous).rate() for d in dates
    ]

In [ ]:
f = plt.figure(figsize=(8, 5))
ax = f.add_subplot(1, 1, 1)

ax.xaxis.grid(True, "major", color="lightgray")
ax.yaxis.grid(True, "major", color="lightgray")
ax.xaxis.set_major_locator(YearLocator(10))
ax.set_ylim(0.03,0.055)

ax.yaxis.set_major_formatter(PercentFormatter(1.0))
styles = iter(["-", "--", ":", "-."])

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="curve_yield",
    ax=ax
)

sns.scatterplot(
    data=gilts_list,
    x="Latest redemption date",
    y="yield",
    ax=ax
)

### Bank of England Portfolio

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe['Operation date'] = pd.to_datetime(df_boe['Operation date'], format='%d-%m-%Y')
df_boe.set_index('Operation date', inplace=True)

df_boe_exit = df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe_exit['Operation date'] = pd.to_datetime(df_boe_exit['Operation date'], format='%d-%m-%Y')
df_boe_exit.set_index('Operation date', inplace=True)

In [ ]:
gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)

gilt_buys_ts = gilt_level_buys.pivot_table(
    index='Operation date', 
    columns='ISIN', 
    values='Total allocation (nominal £mn)', aggfunc='sum')

gilt_buys_ts = gilt_buys_ts.fillna(0).cumsum()

gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)


In [ ]:
gilt_buys_total.head()

In [ ]:
gilts_list.head()

In [ ]:
gilts_list.shape

In [ ]:
gilts_list = gilts_list.merge(gilt_buys_total, left_on='ISIN Code', right_on='ISIN', how='left')

In [ ]:
gilts_list.shape

In [ ]:
gilts_list.columns

In [ ]:
gilt_level_sales = df_sell[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_sales.set_index('Operation date', inplace=True)
gilt_level_sales.index = pd.to_datetime(gilt_level_sales.index, format='%d-%m-%Y')
gilt_level_sales.reset_index(inplace=True)
gilt_level_sales.head()

In [ ]:
gilt_sales_date = gilt_level_sales.loc[gilt_level_sales['Operation date']< calculation_date]

In [ ]:
gilt_sales_date.groupby(['ISIN']).agg(
    Total_Sales_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Sales_Nominal=('Total allocation (nominal £mn)', 'sum'))

In [ ]:
gilts_list = gilts_list.merge(
    gilt_sales_date.groupby(['ISIN']).agg(
    Total_Sales_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Sales_Nominal=('Total allocation (nominal £mn)', 'sum')
    ), 
    left_on='ISIN', 
    right_on='ISIN', 
    how='left')

In [ ]:
gilts_list.head()

In [ ]:
gilts_list['Total_Sales_Proceeds'].sum()
gilts_list['Amount_Outstanding']=gilts_list['Sequence'].map( lambda x: conv.loc[calculation_date, x])

In [ ]:
gilts_list = gilts_list.fillna(0)

In [ ]:
focus = gilts_list.loc[(gilts_list['segmentation']==1)&(gilts_list['price']<100)].copy()

In [ ]:
funding_rate = base_rate.loc[base_rate.index==calculation_date]['Rate'][0]

In [ ]:
# interest paid on the bonds
(focus['Amount_Outstanding']*focus['%']/100).sum()

In [ ]:
# interest received by boe
((focus['Total_Allocation_Nominal'] - focus['Total_Sales_Nominal'])*focus['%']/100).sum()

In [ ]:
# funding cost of the boe
((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])*funding_rate/100).sum()

In [ ]:
total_cost = (focus['Amount_Outstanding']*focus['%']/100).sum() \
- ((focus['Total_Allocation_Nominal'] - focus['Total_Sales_Nominal'])*focus['%']/100).sum() \
+ ((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])*funding_rate/100).sum()
total_cost

In [ ]:
# market holdings
((focus['Amount_Outstanding'] - focus['Total_Allocation_Nominal'] + focus['Total_Sales_Nominal'])*focus['price']/100).sum()

In [ ]:
# boe holdings
((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])).sum()

In [ ]:
(
    (
        (
            focus['Amount_Outstanding'] - focus['Total_Allocation_Nominal'] + focus['Total_Sales_Nominal']
        )*focus['price']/100 + (
            focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds']
            )
            )*focus['curve_yield']
            ).sum()

In [ ]:
((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])*focus['curve_yield']).sum()

In [ ]:
lme_cost = (
    (
        (
            focus['Amount_Outstanding'] - focus['Total_Allocation_Nominal'] + focus['Total_Sales_Nominal']
        )*focus['price']/100 + (
            focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds']
            )
            )*focus['curve_yield']
            ).sum() \
     - ((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])*focus['curve_yield']).sum() \
     + ((focus['Total_Allocation_Proceeds'] - focus['Total_Sales_Proceeds'])*funding_rate/100).sum()

lme_cost